# LLM Iteration 17

**Aim**

Provide an option for running the model inside the StrokeWard class and a function for running multiple replications.

**Prompt**

Update the model to create a function inside the class that can run the model. Produce a separate function that does a single run of the model and another function that can output multiple replications of the single run function. The summary statistics should be output for each run.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
#### MODIFICATION - code from previous iter ####
import simpy
import numpy as np
import math
import pandas as pd
################################################

In [2]:
#### MODIFICATION - code from previous iter ####
class Scenario:
    def __init__(self):
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self):
        interarrival_distributions = {}
        for patient_type, seed in self.random_seeds.items():
            interarrival_distributions[patient_type] = \
                np.random.default_rng(seed)
        return interarrival_distributions
################################################

To update the model to include functions for running a single simulation run and generating multiple replications with summary statistics for each run, you can add the following methods to the StrokeWard class:

In [3]:
class StrokeWard:
    # Existing code remains unchanged
    #### MODIFICATION - code from previous iter ####
    def __init__(self, env, num_beds, scenario):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.waiting_patients = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }
        self.interarrival_distributions = scenario.create_distributions()
        self.route_probabilities = {
            "Stroke": {"ESD": 0.13, "Rehab Unit": 0.24, "Other": 0.63},
            "TIA": {"ESD": 0.01, "Rehab Unit": 0.01, "Other": 0.98},
            "Complex Neurological": {"ESD": 0.05, "Rehab Unit": 0.11,
                                     "Other": 0.84},
            "Other": {"ESD": 0.10, "Rehab Unit": 0.05, "Other": 0.85}
        }

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.patient_type_store = []
        ##################################

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)
            ##################################

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))
            print(f'Patient {patient.id} ({patient.patient_type}) admitted to '
                  f'the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')
            mu, sigma = calculate_mu_sigma(self.length_of_stay[patient.\
                                           patient_type]["mean"],
                                           self.length_of_stay[patient.\
                                           patient_type]["std_dev"])
            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))
            ##################################
            print(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            route_probabilities = list(self.\
                                       route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())
            
            # Use random seed for patient destination assignment
            rng = np.random.default_rng(patient.seed)
            route_choice = rng.choice(route_names, p=route_probabilities)
            
            print(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')

    def patient_arrival(self, patient_type):
        while True:
            if patient_type in self.interarrival_distributions:
                interarrival_time = self.\
                    interarrival_distributions[patient_type].\
                    exponential(scenario.\
                                default_interarrival_times[patient_type])
                yield self.env.timeout(interarrival_time)
                patient = Patient(self.patient_count, patient_type,
                                  self.\
                                  interarrival_distributions[patient_type])
                self.patient_count += 1
                self.env.process(self.admit_patient(patient))
    ################################################

    def run_model(self, num_replications):
        for _ in range(num_replications):
            self.single_run()

    def single_run(self):
        env = simpy.Environment()
        for patient_type in self.scenario.random_seeds.keys():
            env.process(self.patient_arrival(patient_type))

        # Set the run length to 2 months (default)
        run_length_days = round(60 * 2)

        print("Simulation started...")
        env.run(until=run_length_days)
        print("Simulation ended.")

        self.summary()

    def summary(self):
        mean_wait_time = round(np.mean(self.wait_times), 2)
        mean_waiting_patients = round(np.mean(self.waiting_patients), 2)
        
        summary_data = {
            'Mean Wait Time': [mean_wait_time],
            'Mean Waiting Patients': [mean_waiting_patients]
        }
        
        summary_df = pd.DataFrame(summary_data)
        
        print("Summary:")
        print(summary_df)

# Rest of the code remains unchanged

In [7]:
# Example usage:
env = simpy.Environment()
scenario = Scenario()
#### MODIFICATION - removed num_beds=10 due to error ####
# 'positional argument follows keyword argument'
############################################################
stroke_ward = StrokeWard(env, 10, scenario)

# Run a single simulation
#stroke_ward.single_run()

# Run multiple replications
num_replications = 5
#stroke_ward.run_model(num_replications)

In this updated code snippet:
- The run_model method is introduced to run multiple replications of the model by calling the single_run method for each replication.
- The single_run method sets up and runs a single simulation run with patient arrivals and bed assignments.
- After each run, the summary method is called to calculate and display summary statistics for that specific run.
- The example usage demonstrates running a single simulation and then running multiple replications with summary statistics output for each run.

**Reflection**

Running the code now produces an error: "'StrokeWard' object has no attribute 'scenario'" so has been commented out. This is because, despite specifying that the code remained unchanged from previous iterations, the Scenario class passed to StrokeWard is not set up as a parameter in the class. It did not follow the command to produce a separate function for single_run() and for multiple replications.